# TASK 1: GATHERING REVIEWS

In [409]:
from google_play_scraper import app, reviews_all  # For scraping Google Play Store
import pandas as pd  # For data manipulation and analysis
import numpy as np
from IPython.display import display, HTML  # For displaying data in Jupyter notebook
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For parsing HTML content
import json  # For working with JSON data

In [411]:
def get_app_data(app_id):
    """Fetch app data from Google Play Store"""
    result = app(
        app_id,
        lang='en', # Set language to English
        country='us' # Set country to United States
    )
    return result

def get_app_reviews(app_id, count=1000):
    """Fetch reviews for an app"""
    result = reviews_all(
        app_id,
        lang='en', # Set language to English
        country='us', # Set country to United States
        sleep_milliseconds=0, # No delay between requests
        count=count # Number of reviews to fetch
    )
    return result[:count] # Return only the requested number of reviews

In [413]:
# List of app IDs to scrape
app_ids = [
    'com.SeniorEasyPhone', 'com.senior_safety_phone', 'xyz.arjunsinh.elderlauncher','name.kunes.android.launcher.demo', 'com.uncomplicat.launcher'
]

# Initialize lists to store app data and reviews
app_data_list = []
app_reviews_list = []

# Fetch data for each app
for app_id in app_ids:
    # Use try-except to handle potential errors during data fetching
    # This ensures that if one app fails, the script continues with the next app
    # rather than stopping entirely
    try:
        # Get app data
        data = get_app_data(app_id)
        app_data_list.append(data)

        # Get app reviews
        reviews_data = get_app_reviews(app_id)
        app_reviews_list.append({'app_id': app_id, 'reviews': reviews_data})
       

    except Exception as e:
        print(f"Error fetching data for {app_id}: {e}")

In [414]:
for app_reviews in app_reviews_list:
    # Get the app ID
    app_id = app_reviews['app_id']

    # Iterate through the reviews for each app
    for i, review in enumerate(app_reviews['reviews'], 1):
        print(f"{app_id}, {review['userName']}, {review['content']}, {review['score']}")

com.SeniorEasyPhone, Mary Lawson, Excellent for seniors!, 5
com.SeniorEasyPhone, Glendeen Nelson, Not very sensitive to touch with dry skin on my phone, 2
com.SeniorEasyPhone, Douglas Weller, Could not be more simple but would love to eliminate Google ADS but no way even paying a small sum. How about a Whatsapp icon?, 5
com.SeniorEasyPhone, Anna Jessen, Couldn't figure out how to customize it. I wanted what's app on the home screen and couldn't figure it out., 2
com.SeniorEasyPhone, 様テオ, I cat see now i cat see, 5
com.SeniorEasyPhone, 周先生, Great app.Thanks, 5
com.SeniorEasyPhone, Lawry McCarthy, Won't landscape, 1
com.SeniorEasyPhone, Charles Hamrick, Ads block usefulness, 1
com.SeniorEasyPhone, Dada Bhai, Well try. Not in reality not useful, 2
com.SeniorEasyPhone, Yusuf Christopher, The app is OK, but something is missing when you dial number directly from the dial number it is not connecting at all just freeze and the ads need to be remove., 3
com.SeniorEasyPhone, Donald Brown Jr., T

# TASK 2: PREPROCESS YOUR TEXT

In [416]:
import re
import emoji
import nltk
from num2words import num2words
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikhilnambiarchengat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nikhilnambiarchengat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [417]:
def convert_numbers_in_text(text):
    """Replace numbers in the text with their word equivalents."""
    return re.sub(r'\b\d+\b', lambda match: num2words(int(match.group())), text)

def clean_whitespace(text):
    """Remove leading/trailing whitespace and replace multiple spaces with one."""
    return re.sub(r'\s+', ' ', text.strip())

def remove_stop_words(text):
    """Remove stop words from the text."""
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

def lemmatize_review(text):
    """Lemmatize the words in the text."""
    return ' '.join([lemmatizer.lemmatize(word.lower()) for word in text.split() if word.isalpha()])

# List to store processed reviews
processed_reviews = []
package_names = []
for app_reviews in app_reviews_list:
    for review in app_reviews['reviews']:
        if review and review.get('content'):
            review_unprocessed = review['content']
        
            # Remove emojis and special characters, convert numbers to words, remove stop words, etc.
            processed_review = emoji.replace_emoji(review_unprocessed, replace='')  # Remove emojis
            processed_review = re.sub(r'[^A-Za-z0-9\s]', '', processed_review)  # Remove punctuation, special characters
            processed_review = convert_numbers_in_text(processed_review)  # Convert numbers to words
            processed_review = clean_whitespace(processed_review)  # Clean up extra spaces
            processed_review = processed_review.lower()  # Convert to lowercase
            processed_review = remove_stop_words(processed_review)  # Remove stop words
            processed_review = lemmatize_review(processed_review)  # Lemmatize the text
        
            # Append the processed review to the list
            processed_reviews.append(processed_review)
            package_names.append(app_reviews['app_id'])
        else:
            processed_reviews.append('')
            package_names.append('')
            
    
# Print only the first 20 processed reviews
for i, review in enumerate(processed_reviews[:250], ):
    print(review)


excellent senior
sensitive touch dry skin phone
could simple would love eliminate google ad way even paying small sum whatsapp icon
couldnt figure customize wanted whats app home screen couldnt figure
cat see cat see
great appthanks
wont landscape
ad block usefulness
well try reality useful
app ok something missing dial number directly dial number connecting freeze ad need remove
first app look like work old uncle ad prevent functionality really pass
defeat purpose app older peoplepeople hard time using phone litter app popup ad deceitful irresponsible wanted something good grandfather use move yall suck god
restart phone get screen total rubbish
ad way
confusing operate
large contact button open std tiny contact screen unsuitable

annoying ad get way icon something help people require paying money use fully
two change one remove ad two ability edit app shown main scree
good
dificult use dont waste time
app old people dont know use smart phone developer decided add bottom screen time c

# TASK 3: SENTIMENT ANALYSIS

In [419]:
from textblob import TextBlob

In [425]:
def analyze_sentiment_textblob(text):
    # Create a TextBlob object
    blob = TextBlob(text)
    
    # Get sentiment polarity
    polarity = blob.sentiment.polarity
    
    return polarity
    
review_polarity_textblob = []


for package_name, processed_review in zip(package_names, processed_reviews):
    polarity = analyze_sentiment_textblob(processed_review)
    review_polarity_textblob.append([package_name, processed_review, polarity])

df = pd.DataFrame(review_polarity_textblob, columns=[ 'App Package Name','Review', 'Polarity'])
print(df.head(250)) 

                     App Package Name  \
0                 com.SeniorEasyPhone   
1                 com.SeniorEasyPhone   
2                 com.SeniorEasyPhone   
3                 com.SeniorEasyPhone   
4                 com.SeniorEasyPhone   
..                                ...   
245  name.kunes.android.launcher.demo   
246  name.kunes.android.launcher.demo   
247  name.kunes.android.launcher.demo   
248  name.kunes.android.launcher.demo   
249  name.kunes.android.launcher.demo   

                                                Review  Polarity  
0                                     excellent senior  1.000000  
1                       sensitive touch dry skin phone  0.016667  
2    could simple would love eliminate google ad wa...  0.083333  
3    couldnt figure customize wanted whats app home...  0.000000  
4                                      cat see cat see  0.000000  
..                                                 ...       ...  
245  love pretty well made suite expen

In [427]:
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nikhilnambiarchengat/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [429]:
def analyze_sentiment_vader(text):
    
    # Get sentiment polarity
    polarity = sia.polarity_scores(text)['compound']
    
    return polarity
    
review_polarity_vader = []


for package_name, processed_review in zip(package_names, processed_reviews):
    polarity = analyze_sentiment_vader(processed_review)
    review_polarity_vader.append([package_name, processed_review, polarity])


df = pd.DataFrame(review_polarity_vader, columns=[ 'App Package Name','Review', 'Polarity'])
print(df.head(250)) 

                     App Package Name  \
0                 com.SeniorEasyPhone   
1                 com.SeniorEasyPhone   
2                 com.SeniorEasyPhone   
3                 com.SeniorEasyPhone   
4                 com.SeniorEasyPhone   
..                                ...   
245  name.kunes.android.launcher.demo   
246  name.kunes.android.launcher.demo   
247  name.kunes.android.launcher.demo   
248  name.kunes.android.launcher.demo   
249  name.kunes.android.launcher.demo   

                                                Review  Polarity  
0                                     excellent senior    0.5719  
1                       sensitive touch dry skin phone    0.0000  
2    could simple would love eliminate google ad wa...    0.6369  
3    couldnt figure customize wanted whats app home...    0.0000  
4                                      cat see cat see    0.0000  
..                                                 ...       ...  
245  love pretty well made suite expen

### III.

Q: How does the sentiments retrieved by Textblob and Vader compare with eachother? How do they compare with the rating of the app? How do you interpret the similarities/differences?

A: The sentiments retrieved by Textblob and Vader are very different from each other for most reviews, however, they do share similar polarity scores when it comes to reviews that do not make literal sense, those kind of reviews have scored a flat polarity scores of 0 from TextBlob and Vader. Comparing the sentiments retrieved by Textblob and Vader to the user's rating of the app, there doesn't seem to be clear cut relationship between them. A connection that I could make was that the reviews that did score a flat score of 0 for its polarity does seem to have been given a lower rating by the user. 

In [ ]:
GROQ_API_KEY = "API_KEY"

In [435]:
import os
from groq import Groq
from time import sleep

In [ ]:
client = Groq(api_key="API_KEY")

def get_completion(prompt, model="llama3-8b-8192", temperature=0):
 
    try:
        # Make API call to OpenAI's chat completion endpoint
        response = client.chat.completions.create(
            model=model,           # Specify which model to use (e.g., gpt-3.5-turbo)
            messages=[             # Format prompt as a message array
                {
                    "role": "user",    # Set message role as user
                    "content": prompt  # providing actual prompt here
                }
            ],
            temperature=temperature    # Control response randomness (0=focused, 1=creative)
        )
        # Extract and return just the message content from the first response
        return response.choices[0].message.content
    except Exception as e:
        # Handle any API errors and print the error message
        print(f"An error occurred: {e}")
        return None




In [439]:
prompt = """
Classify the sentiment of the following pre-processed reviews on a scale of -1 (negative) to 1 (positive), with 0 representing neutral sentiment:

excellent senior
sensitive touch dry skin phone
could simple would love eliminate google ad way even paying small sum whatsapp icon
couldnt figure customize wanted whats app home screen couldnt figure
cat see cat see
love pretty well made suite expensive almost cheaper buy whole dumb phone senior paying suite hope would revise pricing especially whole suite
love make life easier
lie
havent used much think tremendous potential particularly helping elderly people feel comfortable technology
latest update made possible grandma exit app get lost every time us phone working perfect recent update fix

Please put them in a tabular form
"""
print(prompt)
response = get_completion(prompt)
print(response)


Classify the sentiment of the following pre-processed reviews on a scale of -1 (negative) to 1 (positive), with 0 representing neutral sentiment:

excellent senior
sensitive touch dry skin phone
could simple would love eliminate google ad way even paying small sum whatsapp icon
couldnt figure customize wanted whats app home screen couldnt figure
cat see cat see
love pretty well made suite expensive almost cheaper buy whole dumb phone senior paying suite hope would revise pricing especially whole suite
love make life easier
lie
havent used much think tremendous potential particularly helping elderly people feel comfortable technology
latest update made possible grandma exit app get lost every time us phone working perfect recent update fix

Please put them in a tabular form

Here are the sentiment classifications for each review:

| Review | Sentiment Score |
| --- | --- |
| excellent senior | 1 |
| sensitive touch dry skin phone | -1 |
| could simple would love eliminate google ad way

### Questions:

a) The type of prompt that I used was zero-shot prompting and the prompt that I used was:

"Classify the sentiment of the following pre-processed reviews on a scale of -1 (negative) to 1 (positive), with 0 representing neutral sentiment:

excellent senior
sensitive touch dry skin phone
could simple would love eliminate google ad way even paying small sum whatsapp icon
couldnt figure customize wanted whats app home screen couldnt figure
cat see cat see
love pretty well made suite expensive almost cheaper buy whole dumb phone senior paying suite hope would revise pricing especially whole suite
love make life easier
lie
havent used much think tremendous potential particularly helping elderly people feel comfortable technology
latest update made possible grandma exit app get lost every time us phone working perfect recent update fix

Please put them in a tabular form"

b) Comparing Sentiments returned by Textblob and Groq

| Review                                                              | TextBlob Sentiment | Groq Sentiment |
|----------------------------------------------------------------------|--------------------|-----------------|
| excellent senior                                                     | 1.000000           | 1               |
| sensitive touch dry skin phone                                        | 0.016667           | -1              |
| could simple would love eliminate google ad way even paying small sum whatsapp icon | 0.083333           | 1               |
| couldnt figure customize wanted whats app home screen couldnt figure | 0.000000           | -1              |
| cat see cat see                                                       | 0.000000           | 0               |
| love pretty well made suite expensive almost cheaper buy whole dumb phone senior paying suite hope would revise pricing especially whole suite | 0.079167           | 1               |
| love make life easier                                                 | 0.500000           | 1               |
| lie                                                                   | 0.000000           | -1              |
| havent used much think tremendous potential particularly helping elderly people feel comfortable technology | 0.220000           | 1               |
| latest update made possible grandma exit app get lost every time us phone working perfect recent update fix | 0.375000           | 1               |

Since the prompt explicitly instructed Groq to categroize the reviews either positive, negative or neutral in terms of 1, -1 or 0 respectively, Groq's sentiment analysis is pretty straightforward in comparison to TextBlob's sentiment, which consists of 6 decimal places and random decimals and aren't as clear cut as Groq's sentiment analysis.

c) Comparing Sentiments returned by Vader and Groq

| Review                                                              | Vader Sentiment | Groq Sentiment |
|----------------------------------------------------------------------|--------------------|-----------------|
| excellent senior                                                     | 0.5719          | 1               |
| sensitive touch dry skin phone                                        | 0.0000            | -1              |
| could simple would love eliminate google ad way even paying small sum whatsapp icon | 0.6369             | 1               |
| couldnt figure customize wanted whats app home screen couldnt figure | 0.0000          | -1              |
| cat see cat see                                                       | 0.0000             | 0               |
| love pretty well made suite expensive almost cheaper buy whole dumb phone senior paying suite hope would revise pricing especially whole suite | 0.8442           | 1               |
| love make life easier                                                 | 0.7906           | 1               |
| lie                                                                   | 0.0000           | -1              |
| havent used much think tremendous potential particularly helping elderly people feel comfortable technology | 0.6997           | 1               |
| latest update made possible grandma exit app get lost every time us phone working perfect recent update fix | 0.3400           | 1               |

Since the prompt explicitly instructed Groq to categroize the reviews either positive, negative or neutral in terms of 1, -1 or 0 respectively, Groq's sentiment analysis is pretty straightforward in comparison to Vader's sentiment as well, which consists of 4 decimal places and random decimals and aren't as clear cut as Groq's sentiment analysis.

# TASK 4: TOPIC MODELING Using LDA

In [444]:
from nltk.tokenize import word_tokenize
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

In [446]:
tokenized_reviews = [processed_review.split() for processed_review in processed_reviews]

dictionary = Dictionary(tokenized_reviews)
corpus = [dictionary.doc2bow(tokenized_review) for tokenized_review in tokenized_reviews]

# Train the LDA Model
num_topics = 15
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, passes=10)

topics = lda_model.show_topics(num_topics=num_topics, num_words=7, formatted=False)
for idx, topic in topics:
    print(f"Topic {idx + 1}: {[word for word, _ in topic]}")

Topic 1: ['like', 'app', 'apps', 'cannot', 'worth', 'hate', 'im']
Topic 2: ['call', 'app', 'make', 'version', 'text', 'simple', 'paid']
Topic 3: ['launcher', 'app', 'use', 'contact', 'easy', 'nice', 'screen']
Topic 4: ['like', 'app', 'great', 'launcher', 'show', 'get', 'button']
Topic 5: ['ok', 'app', 'dial', 'bad', 'thanks', 'number', 'call']
Topic 6: ['app', 'call', 'excellent', 'use', 'screen', 'working', 'apps']
Topic 7: ['one', 'app', 'phone', 'version', 'call', 'would', 'use']
Topic 8: ['great', 'useful', 'senior', 'ct', 'using', 'ft', 'fix']
Topic 9: ['app', 'best', 'would', 'use', 'good', 'blind', 'free']
Topic 10: ['good', 'app', 'nice', 'would', 'could', 'old', 'like']
Topic 11: ['app', 'version', 'full', 'phone', 'use', 'launcher', 'big']
Topic 12: ['phone', 'app', 'use', 'big', 'love', 'contact', 'full']
Topic 13: ['please', 'like', 'option', 'thanks', 'work', 'would', 'launcher']
Topic 14: ['app', 'great', 'phone', 'work', 'one', 'need', 'option']
Topic 15: ['app', 'apps',

### II.

Q: How does these review topics (i.e. summarized user feedback) relate to the software features (functionalities) you extracted from app description in your assignment?

A: Some of these review topics does match up with the functionalities or software features that I had extracted from the app description in the assignment. Below are some of the topics that matched up with the features of the app. 
| Topic                                                              | Word | Feature |
|----------------------------------------------------------------------|--------------------|-----------------|
|Topic 2|Simple|Easy to understand and simplified User Experience|
|Topic 3|Easy|Easier way to access applications|
|Topic 3|Easy|Easy to understand and simplified User Experience|
|Topic 11/Topic 12|Big|Large application icons for the stock applications on the home screen|
|Topic 11/Topic 12|Big|Large font size for date and time|


### III.

Q: How does the topics compare to the topics extracted from reviews of each app (SUD + similar/competitor which had review)?

A: 

### IV.

### V.

Q. Specify a list of five Reviews that you believe developers should take care of with
high priority. Discuss your logic for this selection.

A: The list of five reviews that I believe developers should take care of with high priority are:
| Review                                                             | Reasoning|
|----------------------------------------------------------------------|--------------------|
|"Defeats the purpose. If the app is for older people/people that have a hard time using a phone.... then why litter the app for popup ads? Its deceitful and irresponsible. I wanted something good for my grandfather to use and this is NOT the move. Y'all suck on God"|The Advertisement problem is something the developers should take care of, as soon as possible because there are many user reviews complaining about the pestring and intrusive ads. Ads are a way to monetize your application but given this app is built with the intention of helping the elderly, pop-up ads does quite the opposite and can confuse or decieve them. The developers might want to re-think how they implement ads in their app.|
|"Easy layout. Ease of access is great. However, would like it to have a weather widget and dark mode option. Possibly add a high contrast option (like blue and yellow) for visually impaired people like me. Thanks"| Developers should consider prioritizing features like the ones suggested in this user review because the elderly have poor vision as they age. Ease of accessibility is one of the pillars of apps that aim to help the seniors, therefore, the developers must consider this review pretty seriously.|
|"Well, I used to have this on my mom's phone, and it was great, but now it just crashes. Wrote support, never heard back, so 1 star feels generous at this point. Update: finally heard back from the developer. As it turns out they haven't kept up with Android updates from months ago."|Reliability and optimization should be at the forefront when developing an application, especially one that is designed for helping senior citizens. There were a good amount of reviews from users where they were complaining about the poor reliability of the application and since this review got a response back from the developer citing that lack of optimization for the newer updates is the reason behind the poor reliability, it indicates that the developers are doing a poor job maintaining their application. Developers must consider strengthening the maintainability of their application.|
|"I think there is an unnecessarily large number of steps required to call a contact. First you select the person you want to contact (that's fine), then you select call (as opposed to send a text message).At that point, the person's number should be called. But instead, confusingly, there is yet another step in which you have to elect to call the person."|The basis of this application was to simplify the experience of using a cellphone for the senior citizens. Judging just based on this review, it seems like the developers have unintentionally done quite the opposite  and made the basic operations of cellphone, like making a call, more complicated. The developers must prioritize and re-think and refactor their implementation so that they can achieve a more simplified user experience.|
|"PLEASE MAKE YOUR APP'S SETTINGS ICON PASSWORD PROTECTED! Using latest paid version. Absolutely need to lock your settings. My elderly brother accidentally creates havoc by stumbling into Settings and then other screens. He only needs to make and receive calls. Simple. Let's lock him out of everything else, please!"| The developer should look into this feature request and consider prirotizing this user review because being able to easily customize is a core user requirement. However, if whatever customizations the user did can easily or accidentally be overridden, then it defeats the purpose of the customization. Therfore, if a way to lock and save the customaztion can be implemented then, the developer must look into it. |

# TASK 5: GPT AS A STAKEHOLDER TO BRAINSTORM IDEAS FOR ENHANCING YOUR APP’S

a) What type(s) of stakeholder would you consider GPT to be in this context? Explain your reasoning.
A: I'd consider GPT to be Knowledge Stakeholder or a Domain Expert Stakeholder. The reasoning is that GPT has access to large amounts of data about certain things and therefore are vastly knowledgeable about topics that will be beneficial for brainstorming ideas for enhancing the applications.

b) Which requirements engineering techniques would you choose to elicit requirements from GPT for your app? Justify your choice.
A: The requirements engineering techniques that I'd choose to elicit requirements from GPT for the app are as follows:
- brainstorming & design thinking: GPT has access to vast amounts of knowledge and can also get creative with the solutions
- interviews or questionnaires: By giving GPT instructions/prompts to behave like a potential user, it is possible to ask questions and interview GPT like its an actual user.

c) What specific prompt(s) would you use to gather insights and ideas from GPT?
A: Some of the prompts that I'd use to gather insights and ideas from GPT are:
- "Consider yourself as an elderly, who struggles with using a cellphone. Fortunately, a developer is willing to create an application that tries to simplify the experience. What are some of the features you'd like to see in the application?"
- "What are some features that I should include in an application that is meant to help senior citizens use their cellphone more easily?"
- "Lets say you're a developer and you're trying to develop an app that helps seniors use their technology better. However, you have competition as someone already has an app similar to your idea. How would you make your application more unique from a feature set point of view?"

Q: Explain, in detail and with supporting data or examples, how you would use the ideas provided by GPT to enhance your app. Describe both how and why these suggestions would improve your app's functionality.

A: To answer this question, I address the first two reviews from part V of Task IV, to try and enhance my app using ideas suggested by GPT. 
1. Review 1
   Problem: Pop Ups Ads
   
   Prompt: "Lets say you're a developer of an application that is meant to help simplify the cellphone experience for the seniors. One of the complaints right now is the intrusive pop up ads. How would you fix this?"
   
   Suggestions provided by GPT:
   
       a. Offer an Ad-Free Paid Version
       b. Switch to Less Intrusive Ad Formats
       c. Optimize Ad Frequency
       d. Provide Clear Dismissal Options
       e. Implement Contextual and Relevant Ads
       f. Educate Users About Ads


   I'd try implementing suggestions b, d and f to enhance the app. These approaches prioritizes a positive, a less intrusive user experience while maintaining the revenue stream from ads, making the app more accessible and enjoyable for senior users, while making developement of the app sustainable.

2. Review 2
   Problem: Lack of visual accessibility features

   Prompt: "Lets say you're a developer of an application that is meant to help simplify the cellphone experience for the seniors. One of the complaints right now is the lack of visual accessibility features like dark mode or high contrast mode. How would you fix this?"

   Suggestions provided by GPT:
   
       a. Implement Dark Mode
       b. Add a High Contrast Mode
       c. Support Larger Text and Customizable Font Sizes
       d. Include a Color Adjustment Feature
       e. Provide Visual Feedback for Actions

As the developer for the application, I'd try to implement all of these suggestions beacuse by incorporating and developing these features, the app becomes more user-friendly and visually accessible, significantly enhancing its appeal to seniors.